In [15]:
from neo4j import GraphDatabase
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
driver = GraphDatabase.driver("neo4j+s://a71ad3ae.databases.neo4j.io", auth=("neo4j", "JYzNlAikmiFLjlTocP1c15qtT4St9oyIK-ZeIzEKlVo"))

In [17]:
driver.verify_connectivity()

In [18]:
def get_user_room_matrix():
    with driver.session() as session:
        query = """
        MATCH (u:User)-[i:INTERACTED_WITH]->(r:Room)
        RETURN u.id as user_id, r.id as room_id
        """
        result = session.run(query)
        user_ids = []
        room_ids = []
        for record in result:
            user_ids.append(record["user_id"])
            room_ids.append(record["room_id"])

        distinct_users = list(set(user_ids))
        distinct_rooms = list(set(room_ids))

        user_id_map = {user_id: idx for idx, user_id in enumerate(distinct_users)}
        room_id_map = {room_id: idx for idx, room_id in enumerate(distinct_rooms)}

        ur_matrix = np.zeros((len(distinct_users), len(distinct_rooms)))

        for user_id, room_id in zip(user_ids, room_ids):
            user_idx = user_id_map[user_id]
            room_idx = room_id_map[room_id]
            ur_matrix[user_idx, room_idx] = 1

        return ur_matrix, distinct_users, distinct_rooms


In [19]:
def get_similar_users(user_id, ur_matrix, distinct_users):
    target_user_idx = distinct_users.index(user_id)
    similarities = cosine_similarity(ur_matrix[target_user_idx].reshape(1, -1), ur_matrix)[0]
    recommended_users = [(distinct_users[i], similarities[i]) for i in range(len(similarities)) if i != target_user_idx]
    recommended_users.sort(key=lambda x: x[1], reverse=True)
    return recommended_users

In [20]:
def get_recommendations(user_id, similar_users, ur_matrix, distinct_rooms, top_n=5):
    target_user_idx = unique_users.index(user_id)
    target_user_interactions = ur_matrix[target_user_idx]
    recommended_users = []

    for room_idx in range(len(distinct_rooms)):
        if target_user_interactions[room_idx] == 0:
            weighted_sum = 0
            similarity_sum = 0

            for user, similarity in similar_users:
                user_idx = unique_users.index(user)
                weighted_sum += similarity * ur_matrix[user_idx, room_idx]
                similarity_sum += similarity

            if similarity_sum > 0:
                recommendation_score = weighted_sum / similarity_sum
                recommended_users.append((distinct_rooms[room_idx], recommendation_score))

    recommended_users.sort(key=lambda x: x[1], reverse=True)
    return recommended_users[:top_n]

In [28]:
target_user_id = "user20"
user_room_matrix, unique_users, unique_rooms = get_user_room_matrix()
similar_users = get_similar_users(target_user_id, user_room_matrix, unique_users)
recommendations = get_recommendations(target_user_id, similar_users, user_room_matrix, unique_rooms)
print(recommendations)

[('room7', 0.5625258434616128), ('room3', 0.5584570839568203), ('room5', 0.5346156975159639), ('room4', 0.42732751750946374)]
